In [1]:
# Import the dependencies
import pandas as pd
import matplotlib as plt
import numpy as np
from citipy import citipy
import requests
from datetime import datetime

# Import API key
from config import weather_api_key
# url = "api.openweathermap.org/data/2.5/weather?q={city name}&appid=" + weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs


In [3]:
# # Add the lats and longs to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
lat_lngs = []
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

623

In [5]:
# list for cities
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kaduna
Processing Record 2 of Set 1 | arraial do cabo
Processing Record 3 of Set 1 | bereda
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | rocha
Processing Record 6 of Set 1 | upington
Processing Record 7 of Set 1 | huaihua
Processing Record 8 of Set 1 | louisbourg
City not found. Skipping...
Processing Record 9 of Set 1 | samusu
City not found. Skipping...
Processing Record 10 of Set 1 | kuah
Processing Record 11 of Set 1 | victoria
Processing Record 12 of Set 1 | bredasdorp
Processing Record 13 of Set 1 | pevek
Processing Record 14 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 15 of Set 1 | mandurah
Processing Record 16 of Set 1 | norman wells
Processing Record 17 of Set 1 | padang
Processing Record 18 of Set 1 | hilo
Processing Record 19 of Set 1 | reconquista
Processing Record 20 of Set 1 | domodossola
Processing Record 21 of Set 1 | hasaki
P

In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kaduna,10.5222,7.4383,80.17,48,72,5.03,NG,2021-10-16 16:54:18
1,Arraial Do Cabo,-22.9661,-42.0278,78.76,78,75,5.75,BR,2021-10-16 16:54:18
2,Bereda,43.2686,-7.5406,61.41,100,100,1.57,ES,2021-10-16 16:54:19
3,Rikitea,-23.1203,-134.9692,74.77,78,12,9.13,PF,2021-10-16 16:54:19
4,Rocha,-34.4833,-54.3333,59.22,45,2,10.09,UY,2021-10-16 16:54:20
5,Upington,-28.4478,21.2561,68.38,15,0,10.36,ZA,2021-10-16 16:54:20
6,Huaihua,27.5494,109.9592,53.73,90,100,7.72,CN,2021-10-16 16:54:20
7,Kuah,6.3265,99.8432,82.38,89,20,0.00,MY,2021-10-16 16:54:21
8,Victoria,22.2855,114.1577,76.32,70,19,1.99,HK,2021-10-16 16:50:08
9,Bredasdorp,-34.5322,20.0403,55.27,52,91,13.85,ZA,2021-10-16 16:54:22


In [7]:
# Reorder columns
column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[column_order]


In [8]:
# Create the output file CSV
output_data_file = "weather_data/cities.csv"
# Export the City Data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [9]:
# Extract relevant field from the DF for plotting
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]

In [10]:
# Import time module
import time

# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,
            max_temps,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
plt.savefig("weather_data/Fig1.png")

# Show plot.
plt.show()

AttributeError: module 'matplotlib' has no attribute 'scatter'

In [ ]:
# Build the scatter plots for lat vd humidity
plt.scatter(lats, 
            humidity, 
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Humidity "+ time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig2.png")
# Show plot.
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. cloudiness.
plt.scatter(lats,
            cloudiness,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Cloudiness (%) "+ time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig3.png")
# Show plot.
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. wind speed.
plt.scatter(lats,
            wind_speed,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Wind Speed "+ time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig4.png")
# Show plot.
plt.show()